In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split, KFold

from sklearn.metrics import accuracy_score

In [2]:
# loading data
raw_data = loadarff('mnist_784.arff')
df = pd.DataFrame(raw_data[0])

In [3]:
X = df.iloc[:, 0:-1]
y = df.iloc[:, [-1]].astype('int') # target

X, X_val, y, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)


n_classes = 10

# Convert target classes to categorical ones
y = to_categorical(y, n_classes)

kf = KFold(n_splits=10, shuffle=True)

In [4]:
from utils.mlp import MLP
model = MLP(
    activation='relu',
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
    input_shape=(28*28,),
    n_classes=10,
    n_neurons=[500, 500, 500]
)

2022-08-31 14:06:05.376742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 14:06:05.404733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 14:06:05.404894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 14:06:05.405355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
# results = list()
# for fold, (train, test) in enumerate(kf.split(X, y)):
#     X_train, X_test = X.iloc[train].values, X.iloc[test].values
#     y_train, y_test = y[train], y[test]

#     mlp.fit(X_train, y_train, epochs=10, batch_size=128, verbose=0)

#     print(f'Fold {fold}')

#     y_pred = np.argmax(mlp.predict(X_test), axis=1)
#     y_test = np.argmax(y_test, axis=1)
#     acc = accuracy_score(y_test, y_pred)

#     print(f"Accuracy: {acc}")
#     results.append(acc)

# print(f'Accuracy mean (std): {np.mean(results):.3f} ({np.std(results):.3f})')

    

In [9]:
model.fit(X, y, epochs=20, batch_size=256)
y_pred = np.argmax(model.predict(X_val), axis=1)

print(accuracy_score(y_val, y_pred))

Epoch 1/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0532 - accuracy: 0.9865
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0518 - accuracy: 0.9873
Epoch 3/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0605 - accuracy: 0.9852
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0549 - accuracy: 0.9865
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0458 - accuracy: 0.9887
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0480 - accuracy: 0.9881
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0422 - accuracy: 0.9895
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0368 - accuracy: 0.9911
Epoch 9/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0357 - accuracy: 0.9909
Epoch 10/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0434 - accuracy: 0.9888

In [7]:
import random
import cv2

def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

imgs = ['cinco', 'dois', 'nove', 'oito', 'oito(1)', 'tres']
for path in imgs:
    image = cv2.imread(f'imgs/{path}.png',0) # Only for grayscale image
    noise_img = sp_noise(image, 0.1)
    cv2.imwrite(f'noise_imgs/noise_{path}.jpg', noise_img)




In [8]:
from utils.num import Num

for i in imgs:
    n = Num(
        path=f'noise_imgs/noise_{i}.jpg',
        invert=True,
    )

    print(i, np.argmax(mlp.predict(n.to_array())))


NameError: name 'mlp' is not defined